In [2]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta
import numpy as np
from pprint import pprint

# Initialize Faker
fake = Faker()

# SQLite database file
DB_FILE = "medical.db"

# Medical configuration
DIAGNOSES = [
    "Type 2 Diabetes", "Hypertension", "Hyperlipidemia", 
    "Coronary Artery Disease", "Asthma", "COPD",
    "Osteoarthritis", "Depression", "Anxiety Disorder",
    "Hypothyroidism", "GERD", "Chronic Kidney Disease"
]

INSURANCE_PROVIDERS = [
    "Medicare", "Medicaid", "Blue Cross", "Aetna",
    "UnitedHealthcare", "Cigna", "Kaiser Permanente"
]

LAB_TESTS = [
    ("HbA1c", 4.0, 5.6, "%"),
    ("Glucose", 70, 99, "mg/dL"),
    ("LDL", 0, 100, "mg/dL"),
    ("HDL", 40, 60, "mg/dL"),
    ("Triglycerides", 0, 150, "mg/dL"),
    ("Creatinine", 0.5, 1.2, "mg/dL"),
    ("TSH", 0.4, 4.0, "mIU/L"),
    ("WBC", 4.5, 11.0, "K/uL"),
    ("Hemoglobin", 12, 16, "g/dL")
]

COMMON_MEDS = [
    ("Metformin", "500 mg", "BID", "PO"),
    ("Lisinopril", "10 mg", "Daily", "PO"),
    ("Atorvastatin", "40 mg", "HS", "PO"),
    ("Albuterol", "90 mcg", "Q4H PRN", "Inhalation"),
    ("Sertraline", "50 mg", "Daily", "PO"),
    ("Levothyroxine", "50 mcg", "Daily", "PO"),
    ("Omeprazole", "20 mg", "Daily", "PO"),
    ("Losartan", "50 mg", "Daily", "PO"),
    ("Insulin Glargine", "10 units", "HS", "SubQ")
]

def get_db_connection():
    return sqlite3.connect(DB_FILE)

def truncate(value, max_length):
    """Ensure string values don't exceed column limits"""
    return str(value)[:max_length] if value else None

def generate_phone():
    """Generate phone numbers guaranteed to fit in 15 chars"""
    formats = [
        '###-###-####',      # 12 chars
        '(###) ###-####',    # 14 chars
        '+1 ###-###-####'    # 15 chars
    ]
    return fake.numerify(text=random.choice(formats))

def create_tables():
    """Create SQLite database tables"""
    conn = get_db_connection()
    cur = conn.cursor()
    
    commands = [
        "DROP TABLE IF EXISTS medications",
        "DROP TABLE IF EXISTS lab_results",
        "DROP TABLE IF EXISTS visits",
        "DROP TABLE IF EXISTS patients",
        """
        CREATE TABLE patients (
            patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT,
            last_name TEXT,
            birth_date TEXT,
            gender TEXT,
            race TEXT,
            address TEXT,
            city TEXT,
            state TEXT,
            zip_code TEXT,
            phone TEXT,
            email TEXT,
            primary_diagnosis TEXT,
            insurance_provider TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP
        )
        """,
        """
        CREATE TABLE visits (
            visit_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            visit_date TEXT,
            visit_type TEXT,
            diagnosis_code TEXT,
            diagnosis_description TEXT,
            physician TEXT,
            notes TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
        )
        """,
        """
        CREATE TABLE lab_results (
            result_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            visit_id INTEGER,
            test_type TEXT,
            result_value REAL,
            reference_range TEXT,
            result_date TEXT,
            units TEXT,
            abnormal_flag INTEGER,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (patient_id) REFERENCES patients(patient_id),
            FOREIGN KEY (visit_id) REFERENCES visits(visit_id)
        )
        """,
        """
        CREATE TABLE medications (
            medication_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            visit_id INTEGER,
            drug_name TEXT,
            dosage TEXT,
            frequency TEXT,
            route TEXT,
            start_date TEXT,
            end_date TEXT,
            prescribed_by TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (patient_id) REFERENCES patients(patient_id),
            FOREIGN KEY (visit_id) REFERENCES visits(visit_id)
        )
        """
    ]
    
    for command in commands:
        cur.execute(command)
    conn.commit()
    conn.close()
    print("SQLite database tables created successfully")

def generate_patients(num_patients=100):
    """Generate complete patient records with visits, labs, and medications"""
    conn = get_db_connection()
    cur = conn.cursor()
    
    for i in range(1, num_patients + 1):
        # Generate patient
        cur.execute("""
        INSERT INTO patients (
            first_name, last_name, birth_date, gender, race,
            address, city, state, zip_code, phone, email,
            primary_diagnosis, insurance_provider
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            truncate(fake.first_name(), 50),
            truncate(fake.last_name(), 50),
            fake.date_of_birth(minimum_age=18, maximum_age=90).strftime('%Y-%m-%d'),
            truncate(random.choice(["Male", "Female"]), 10),
            truncate(random.choice(["White", "Black", "Asian", "Hispanic", "Other"]), 30),
            truncate(fake.street_address(), 100),
            truncate(fake.city(), 50),
            truncate(fake.state_abbr(), 2),
            truncate(fake.zipcode()[:10], 10),
            generate_phone(),
            truncate(fake.email(), 100),
            truncate(random.choice(DIAGNOSES), 100),
            truncate(random.choice(INSURANCE_PROVIDERS), 50)
        ))
        patient_id = cur.lastrowid
        
        # Generate 1-5 visits per patient
        for _ in range(random.randint(1, 5)):
            visit_date = fake.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d')
            
            cur.execute("""
            INSERT INTO visits (
                patient_id, visit_date, visit_type,
                diagnosis_code, diagnosis_description,
                physician, notes
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                patient_id,
                visit_date,
                truncate(random.choice(["Routine", "Follow-up", "Emergency", "Consult", "Procedure"]), 30),
                truncate(fake.bothify(text='??##'), 10),
                truncate(random.choice(DIAGNOSES), 200),
                truncate(fake.name(), 100),
                fake.text(max_nb_chars=500)
            ))
            visit_id = cur.lastrowid
            
            # Generate 1-3 lab results per visit
            for _ in range(random.randint(1, 3)):
                test_type, low, high, units = random.choice(LAB_TESTS)
                base_value = random.uniform(low, high)
                
                # 20% chance of abnormal result
                if random.random() < 0.2:
                    modifier = random.choice([0.5, 1.5])
                    result_value = round(base_value * modifier, 2)
                else:
                    result_value = round(base_value, 2)
                
                abnormal = int(not (low <= result_value <= high))
                
                cur.execute("""
                INSERT INTO lab_results (
                    patient_id, visit_id, test_type,
                    result_value, reference_range,
                    result_date, units, abnormal_flag
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    patient_id,
                    visit_id,
                    truncate(test_type, 30),
                    result_value,
                    truncate(f"{low}-{high}", 30),
                    visit_date,
                    truncate(units, 10),
                    abnormal
                ))
            
            # Generate 1-4 medications per visit
            for _ in range(random.randint(1, 4)):
                drug_name, dosage, frequency, route = random.choice(COMMON_MEDS)
                start_date = visit_date
                end_date = (datetime.strptime(start_date, '%Y-%m-%d') + 
                           timedelta(days=random.randint(30, 365))).strftime('%Y-%m-%d') if random.random() > 0.3 else None
                
                cur.execute("""
                INSERT INTO medications (
                    patient_id, visit_id, drug_name,
                    dosage, frequency, route,
                    start_date, end_date, prescribed_by
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    patient_id,
                    visit_id,
                    truncate(drug_name, 50),
                    truncate(dosage, 30),
                    truncate(frequency, 30),
                    truncate(route, 20),
                    start_date,
                    end_date,
                    truncate(fake.name(), 100)
                ))
        
        if i % 10 == 0:
            print(f"Generated patient {i}/{num_patients}")
    
    conn.commit()
    conn.close()
    print(f"Successfully generated data for {num_patients} patients")

def verify_data():
    """Verify data was inserted correctly"""
    conn = get_db_connection()
    conn.row_factory = sqlite3.Row  # Enable column access by name
    
    queries = [
        ("Patients", "SELECT COUNT(*) AS count FROM patients"),
        ("Visits", "SELECT COUNT(*) AS count FROM visits"),
        ("Lab Results", "SELECT COUNT(*) AS count FROM lab_results"),
        ("Medications", "SELECT COUNT(*) AS count FROM medications"),
        ("Sample Patient", """
         SELECT p.first_name, p.last_name, p.primary_diagnosis,
                COUNT(DISTINCT v.visit_id) as visits,
                COUNT(DISTINCT l.result_id) as labs,
                COUNT(DISTINCT m.medication_id) as meds
         FROM patients p
         LEFT JOIN visits v ON p.patient_id = v.patient_id
         LEFT JOIN lab_results l ON p.patient_id = l.patient_id
         LEFT JOIN medications m ON p.patient_id = m.patient_id
         GROUP BY p.patient_id
         LIMIT 5
         """)
    ]
    
    for name, query in queries:
        print(f"\n{name}:")
        results = conn.execute(query).fetchall()
        for row in results:
            pprint(dict(row))
    
    conn.close()

# Main execution
if __name__ == "__main__":
    create_tables()
    generate_patients(1000)  # Generate 100 complete patient records
    verify_data()

SQLite database tables created successfully
Generated patient 10/1000
Generated patient 20/1000
Generated patient 30/1000
Generated patient 40/1000
Generated patient 50/1000
Generated patient 60/1000
Generated patient 70/1000
Generated patient 80/1000
Generated patient 90/1000
Generated patient 100/1000
Generated patient 110/1000
Generated patient 120/1000
Generated patient 130/1000
Generated patient 140/1000
Generated patient 150/1000
Generated patient 160/1000
Generated patient 170/1000
Generated patient 180/1000
Generated patient 190/1000
Generated patient 200/1000
Generated patient 210/1000
Generated patient 220/1000
Generated patient 230/1000
Generated patient 240/1000
Generated patient 250/1000
Generated patient 260/1000
Generated patient 270/1000
Generated patient 280/1000
Generated patient 290/1000
Generated patient 300/1000
Generated patient 310/1000
Generated patient 320/1000
Generated patient 330/1000
Generated patient 340/1000
Generated patient 350/1000
Generated patient 36